<a href="https://colab.research.google.com/github/Leo-Kasper/is310-final-project-leo-kasper/blob/main/final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy
import nltk
nltk.download('punkt')
from nltk import word_tokenize
from nltk import FreqDist
from nltk.stem import PorterStemmer
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
porter = PorterStemmer()
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.sentiment import SentimentIntensityAnalyzer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
kazuya = pd.read_csv('kazuya.csv')
bayo = pd.read_csv('bayo.csv')
steve = pd.read_csv('steve.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,1,2,3,6,11,12,15,20,22) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
kazuya.head()

,Unnamed: 0,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,0,1524102744340062212,1524102744340062212,1.652209e+12,2022-05-10 19:03:05,0,NaN,Steve and Kazuya are the future of smash I do ...,en,[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1,1524043507299602434,1524043165199593472,1.652195e+12,2022-05-10 15:07:42,0,NaN,"Speaking of, Smash Ultimate is so weird becaus...",en,[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,4,1523913102965805058,1523913102965805058,1.652164e+12,2022-05-10 06:29:31,0,NaN,I'm just imagining how terrified Smash players...,en,[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,6,1523849081449238528,1523849081449238528,1.652149e+12,2022-05-10 02:15:07,0,NaN,I like how some Smash tourneys will ban Steve ...,en,[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,7,1523847584355438594,1523847584355438594,1.652149e+12,2022-05-10 02:09:10,0,NaN,I want to talk about Kazuya after playing non-...,en,[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [4]:
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [5]:
# bayo_sentiment = 0.0

# bayo[~bayo['tweet'].isnull()]

# for tweet in bayo['tweet']:
#   bayo_sentiment += sia.polarity_scores(tweet)["compound"]

# bayo_sentiment

In [6]:
def stem_words(row):
    stemmed_words = ''
    for token in row.tweet.split(' '):
        stemmed_words += porter.stem(token) + ' '
    return stemmed_words

In [7]:
bayo_clean = pd.DataFrame().assign(tweet=bayo['tweet'], username=bayo['username'])

In [8]:
kazuya_clean = pd.DataFrame().assign(tweet=kazuya['tweet'], username=kazuya['username'])

In [9]:
steve_clean = pd.DataFrame().assign(tweet=steve['tweet'], username=steve['username'])

In [10]:
bayo_clean.head()

,tweet,username
0,"@PowerFuck Dmc 5, smash ultimate uhh bayonetta...",Handler_Chan
1,its so weird seeing non smash twitter talking ...,AceOfGaykes
2,"If there’s a Bayonetta 3, Smash or Metroid Pri...",FinalGrantasy
3,"@Actual_YTWaffle @geoffkeighley Well, you can ...",MATT_HAZARD_
4,"If I end up missing the Game Awards today, I t...",Light_Suit213


In [11]:
bayo_clean = bayo_clean[bayo_clean['tweet'].notnull()]
bayo_clean['stemmed_text'] = bayo_clean.apply(stem_words,axis=1)

In [12]:
kazuya_clean = kazuya_clean[kazuya_clean['tweet'].notnull()]
kazuya_clean['stemmed_text'] = kazuya_clean.apply(stem_words,axis=1)

In [13]:
steve_clean = steve_clean[steve_clean['tweet'].notnull()]
steve_clean['stemmed_text'] = steve_clean.apply(stem_words,axis=1)

In [14]:
bayo_clean

,tweet,username,stemmed_text
0,"@PowerFuck Dmc 5, smash ultimate uhh bayonetta...",Handler_Chan,"@powerfuck dmc 5, smash ultim uhh bayonetta 3 ..."
1,its so weird seeing non smash twitter talking ...,AceOfGaykes,it so weird see non smash twitter talk about s...
2,"If there’s a Bayonetta 3, Smash or Metroid Pri...",FinalGrantasy,"If there’ a bayonetta 3, smash or metroid prim..."
3,"@Actual_YTWaffle @geoffkeighley Well, you can ...",MATT_HAZARD_,"@actual_ytwaffl @geoffkeighley well, you can s..."
4,"If I end up missing the Game Awards today, I t...",Light_Suit213,"If I end up miss the game award today, I think..."
...,...,...,...
26253,My mains #smash #sexyteam #zerosuitsamus #bayo...,AnarchyReira,My main #smash #sexyteam #zerosuitsamu #bayone...
26254,Bayonetta is coming to Smash Bros. tomorrow! #...,PetesuhPan,bayonetta is come to smash bros. tomorrow! #hype
26255,@AppleOfClowns Bayonetta should release for Sm...,Avvvee,@appleofclown bayonetta should releas for smas...
26256,I totally forgot that bayonetta comes out tomo...,MrTacoBelmonto,I total forgot that bayonetta come out tomorro...


In [16]:
bayo_tweets = bayo_clean.tweet.tolist()
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
kazuya_tweets = kazuya_clean.tweet.tolist()
steve_tweets = steve_clean.tweet.tolist()

In [18]:
vectorizer = TfidfVectorizer(max_df=0.5, min_df=1,stop_words=stop_words)

In [ ]:
transformed_bayo_tweets = vectorizer.fit_transform(bayo_tweets)

transformed_bayo_tweets_as_array = transformed_bayo_tweets.toarray()

tfidf_results_b = []
for counter, doc in enumerate(transformed_bayo_tweets_as_array):
    tf_idf_tuples = list(zip(vectorizer.get_feature_names(), doc))
    one_doc_as_df = pd.DataFrame.from_records(tf_idf_tuples, columns=['term', 'score']).sort_values(by='score', ascending=False).reset_index(drop=True)
    tfidf_results_b.append(one_doc_as_df)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
